In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from sklearn.datasets import load_sample_image

china = load_sample_image("china.jpg")/255
flower = load_sample_image("flower.jpg")/255
images = np.array([china,flower])

In [79]:
class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters,3,strides, padding='same', use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters,3,1,padding='same', use_bias=False),
            keras.layers.BatchNormalization(),
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(filters,1, strides,padding='same', use_bias=False),
                keras.layers.BatchNormalization()
            ]
def call(self, inputs):
    Z = inputs
    for layer in self.main_layers:
        Z = layer(Z)
        skip_Z = inputs
    for layer in self.skip_layers:
        skip_Z = layer(skip_Z)
    return self.activation(Z + skip_Z)


In [87]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64,7,2,input_shape=[224,224,3],padding='same'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.ReLU()),
model.add(keras.layers.MaxPool2D(pool_size=3,strides=2,padding='same'))
prev = 64
for i in [64,64,64,128,128,128,128,256,256,256,256,512,512,512]:
    strides = 1 if prev == i else 2
    model.add(ResidualUnit(i,strides,keras.layers.ReLU()))
    prev = i
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())   
model.add(keras.layers.Dense(10,activation='softmax'))

In [20]:
import tensorflow_datasets as tfds
dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
dataset_size = info.splits['train'].num_examples
class_names = info.features['label'].names
n_classes = info.features['label'].num_classes

In [25]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True)

In [26]:
def preprocess(image,label):
    resized_image = tf.image.resize(image, [224,224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label


In [27]:
train_set.shuffle(1000)
train_set = train_set_raw.map(preprocess).batch(32).repeat().prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(32).repeat().prefetch(1)
test_set = test_set_raw.map(preprocess).batch(32).repeat().prefetch(1)

In [17]:
base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation='softmax')(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [28]:
for layer in base_model.layers:
    layer.trainable=False

In [30]:
optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9, decay=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer = optimizer, metrics=["accuracy"])
history = model.fit(train_set,steps_per_epoch=int(0.75 * dataset_size / 32), validation_data = valid_set,validation_steps=int(0.15 * dataset_size / 32), epochs=5)

Epoch 1/5
86/86 [==============================] - 240s 3s/step - loss: 1.1728 - accuracy: 0.8045 - val_loss: 0.8381 - val_accuracy: 0.8382
Epoch 2/5
 5/86 [>.............................] - ETA: 3:01 - loss: 0.5420 - accuracy: 0.9125

KeyboardInterrupt: 

In [32]:
base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
class_output = keras.layers.Dense(n_classes, activation='softmax')(avg)
loc_output = keras.layers.Dense(4)(avg)
model = keras.Model(inputs=base_model.input, outputs=[class_output, loc_output])

In [1]:
model.compile(loss=["sparse_categorical_entropy", "mse"], optimizer = optimizer, metrics=["accuracy"])

NameError: name 'model' is not defined

## MNIST from scratch try Resnet Senet Fully convolutional net, pooling, lrn, etc.

In [92]:
(X_train,y_train),(X_test,y_test) = keras.datasets.fashion_mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28,28,1).astype(np.float32)
X_test = X_test.reshape(X_test.shape[0], 28,28,1).astype(np.float32)


In [93]:
X_train = X_train/255
X_test = X_test/255

In [94]:
model = keras.models.Sequential([
    keras.layers.Conv2D(64,4,2,padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.MaxPool2D(pool_size=3,strides=2,padding='same'),
    ResidualUnit(64,1,keras.layers.ReLU()),
    ResidualUnit(64,1,keras.layers.ReLU()),
    ResidualUnit(64,1,keras.layers.ReLU()),
    ResidualUnit(128,2,keras.layers.ReLU()),
    ResidualUnit(128,2,keras.layers.ReLU()),
    ResidualUnit(128,2,keras.layers.ReLU()),
    ResidualUnit(128,2,keras.layers.ReLU()),
    ResidualUnit(256,2,keras.layers.ReLU()),
    ResidualUnit(256,2,keras.layers.ReLU()),
    ResidualUnit(256,2,keras.layers.ReLU()),
    ResidualUnit(512,2,keras.layers.ReLU()),
    ResidualUnit(512,2,keras.layers.ReLU()),
    ResidualUnit(512,2,keras.layers.ReLU()),
    keras.layers.GlobalAvgPool2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(10,activation="softmax")
])

In [95]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer = keras.optimizers.Adam(learning_rate = 0.001), metrics = ['accuracy'])
model.fit(X_train,y_train,epochs=1000,validation_data = (X_test,y_test),batch_size=32, steps_per_epoch=60000//32)


Epoch 1/1000
1875/1875 [==============================] - 23s 12ms/step - loss: 1.3116 - accuracy: 0.5717 - val_loss: 1.1445 - val_accuracy: 0.6024
Epoch 2/1000
1875/1875 [==============================] - 22s 12ms/step - loss: 0.9911 - accuracy: 0.6685 - val_loss: 0.9925 - val_accuracy: 0.6314
Epoch 3/1000
1875/1875 [==============================] - 26s 14ms/step - loss: 0.8891 - accuracy: 0.7029 - val_loss: 0.9893 - val_accuracy: 0.6490
Epoch 4/1000
1875/1875 [==============================] - 27s 15ms/step - loss: 0.8317 - accuracy: 0.7193 - val_loss: 0.8216 - val_accuracy: 0.7166
Epoch 5/1000
1875/1875 [==============================] - 27s 15ms/step - loss: 0.7942 - accuracy: 0.7329 - val_loss: 0.9803 - val_accuracy: 0.6543
Epoch 6/1000
1875/1875 [==============================] - 36s 19ms/step - loss: 0.7673 - accuracy: 0.7374 - val_loss: 0.7799 - val_accuracy: 0.7215
Epoch 7/1000
1875/1875 [==============================] - 36s 19ms/step - loss: 0.7487 - accuracy: 0.7452 - val_